In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
customers = pd.read_csv('Customers.csv')
transactions = pd.read_csv('Transactions.csv')
products = pd.read_csv('Products.csv')

In [ ]:
merged_df = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')


In [ ]:
print(merged_df.columns)


Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'Price_y'],
      dtype='object')


In [ ]:
customer_features = merged_df.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'Price_x': 'mean',
    'Category': lambda x: x.mode()[0],
    'Region': lambda x: x.mode()[0]
}).reset_index()


In [ ]:
customer_features = pd.get_dummies(customer_features, columns=['Category', 'Region'], drop_first=True)


In [ ]:
scaler = StandardScaler()
customer_features_scaled = scaler.fit_transform(customer_features.drop(columns=['CustomerID']))


In [ ]:
similarity_matrix = cosine_similarity(customer_features_scaled)


In [ ]:
def get_top_3_lookalikes(customer_id, customer_features_df, similarity_matrix):
    index = customer_features_df[customer_features_df['CustomerID'] == customer_id].index[0]
    sim_scores = list(enumerate(similarity_matrix[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_3_customers = [(customer_features_df.iloc[i[0]]['CustomerID'], round(i[1], 2)) for i in sim_scores[1:4]]
    return top_3_customers


In [ ]:
lookalike_results = {}

for cust_id in customers['CustomerID'][:20]:
    lookalike_results[cust_id] = get_top_3_lookalikes(cust_id, customer_features, similarity_matrix)


In [ ]:
lookalike_df = pd.DataFrame.from_dict(lookalike_results, orient='index', columns=['Lookalike1', 'Lookalike2', 'Lookalike3'])
lookalike_df.to_csv('Siripina_Gopi_Lookalike.csv')

print("Lookalike recommendations saved to Siripina_Gopi_Lookalike.csv")


Lookalike recommendations saved to Siripina_Gopi_Lookalike.csv
